# 1.0 Get the raw news information
* 本篇代码用来获取相关公司的新闻文本
* 由于信息渠道广泛，将会爬虫数个新闻网站: [新浪新闻，微信公众号]

### 1.1 新浪新闻获取

In [1]:
import pandas as pd
import xlwt as xlwt

print("Hello, news")
from datetime import datetime
now=datetime.now()
from lxml import etree
import numpy as np
from pylab import mpl
# todo: solve chinese problem for plt
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import json
print("Load time ",datetime.now()-now)

Hello, news
Load time  0:00:00.625339


In [2]:
from requests import RequestException


class newsToolForSina:
    @staticmethod
    def get_list(url):

        # 新闻链接
        res=requests.get(url)
        res.encoding='utf-8'

        # 完整HTML
        html=BeautifulSoup(res.text,'html.parser')

        # 新闻列表
        newList=[]

        for item in html.find_all('div',['news-item','img-news-item']):
            try:
                newObj={}
                newObj['title']=item.select('h2 a')[0].text
                newObj['url']=item.select('h2 a')[0].get('href')
                newList.append(newObj)
            except:
                print('出现异常')
        return newList

    @staticmethod
    def get_HtmlDetail_Sina(url):

        # 新闻链接
        res=requests.get(url)
        res.encoding='utf-8'

        # 完整HTML
        html=BeautifulSoup(res.text,'html.parser')

        # 新闻对象
        result={}

        # 新闻标题
        result['title']=html.select('.main-title')[0].text

        # 发布时间
        timesource=html.select('.date-source span')[0].text
        createtime=datetime.strptime(timesource,'%Y年%m月%d日 %H:%M')
        createtime.strftime('%Y-%m-%d')
        result['createtime']=createtime

        # 新闻来源
        result['place']=html.select('.date-source a')[0].text

        # 新闻内容
        article=[]
        for p in html.select('#article p')[:-1]:
            article.append(p.text.strip())
        articleText=' '.join(article)
        result['article']=articleText

        # 新闻作者
        result['author']=html.select('.show_author')[0].text.strip('责任编辑：')

        # 新闻链接
        result['url']=url

        return result

    @staticmethod
    def get_page_news(browser,urlList):
        #获取当前页面所有包含新闻的a标签
        news = browser.find_elements("xpath",'//div[@class="d_list_txt"]/ul/li/span/a')
        if len(news)==0:
            return np.nan
        for i in news:
            link = i.get_attribute('href') #得到新闻url
            # print(len(urlList),link)
            if link not in urlList:  #通过url去重
                urlList.append(link)
        return urlList

    @staticmethod
    def getNewsData(url):
        # 获取新闻的详细信息
        html = newsToolForSina.get_response(url)
        #使用beautifulsoup进行解析
        soup = BeautifulSoup(html,'lxml')

        #标题
        '''
        <h1 class="main-title">证监会要求北京银行说明是否串通*ST康得管理层舞弊</h1>
        '''
        title = soup.select('.main-title')
        #可能有小部分标题的标签不是上述格式 对其进行补充
        if not title:
            title = soup.select('#artibodyTitle')
        if title:
            title = title[0].text
        # print(title)

        #日期
        '''
        <span class="date">2019年07月20日 16:52</span>
        '''
        date = soup.select('.date')
        # 可能有小部分日期的标签不是上述格式 对其进行补充
        if not date:
            date = soup.select('#pub_date')
        if date:
            date = date[0].text
        # print(date)

        #来源
        '''
        <span class="source ent-source">中国证券报</span>
        '''
        source = soup.select('.source')
        # 可能有小部分来源的标签不是上述格式 对其进行补充
        if not source:
            source = soup.select('[data-sudaclick="media_name"]')
        if source:
            source = source[0].text
        # print(source)

        #正文
        article = soup.select('div[class="article"] p')
        # 可能有小部分正文的标签不是上述格式 对其进行补充
        if not article:
            article = soup.select('div[id="artibody"] p')
        if article:
            #把正文放在一个列表中 每个p标签的内容为列表的一项
            article_list = []
            for i in article:
                # print(i.text)
                article_list.append(i.text)
        #转为字典格式
        news = {'link': url, 'title': title, 'date': date, 'source': source, 'article': article_list}
        return news

    @staticmethod
    def get_response(url):
        try:
            #添加User-Agent，放在headers中，伪装成浏览器
            headers = {
                'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
            }
            response = requests.get(url,headers=headers)
            if response.status_code == 200:
                response.encoding = 'utf-8'
                return response.text
            return None
        except RequestException:
            return None


In [3]:
###test
# newList=newsToolForSina.get_list('https://news.sina.com.cn/world/')
# resultList=pd.DataFrame()
#
# for i,item in enumerate(newList):
#     try:
#         result=newsToolForSina.get_HtmlDetail_Sina(item['url'])
#         newObj=pd.DataFrame(result,index=[0])
#         resultList=resultList.append(newObj)
#         print (str(i),'写入成功')
#     except BaseException as err:
#         print (str(i),'出现异常: ',err)
#
# resultList.to_csv("newsData/test.csv",header=resultList.columns)
# resultList

## 注意: 新浪新闻的历史数据只能查到近期一个月的，并不能查之前的，所以新浪的可以放弃一部分了

In [4]:
# from selenium.common import NoSuchElementException
#
# #打开浏览器
# UrlList=[]
# browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# browser.implicitly_wait(10)
# #打开网址
# browser.get('https://news.sina.com.cn/roll/')
# #获取当前页面新闻的url
# UrlList=newsToolForSina.get_page_news(browser,urlList=UrlList)
# while True:
#     try:
#         #找到下一页按钮 并点击
#         '''
#         <a href="javascript:void(0)" onclick="newsList.page.next();return false;">下一页</a>
#         '''
#         browser.find_element("xpath",'//a[@onclick="newsList.page.next();return false;"]').click()
#         #获取下一页新闻的url
#         result=newsToolForSina.get_page_news(browser,urlList=UrlList)
#         if result is np.nan:
#             break
#     except BaseException as err:
#         print(err)
#         browser.close()
#
#
# print("Have found {} URL records, trying to get news text data".format(len(UrlList)))
# resultList=pd.DataFrame()
# for i,item in enumerate(UrlList):
#     try:
#         result=newsToolForSina.getNewsData(item)
#         newObj=pd.DataFrame(result)
#         resultList=resultList.append(newObj)
#         print (str(i),'写入成功')
#         # break
#     except BaseException as err:
#         print (str(i),'出现异常: ',err)
#         # break
#
#
# resultList.to_csv("newsData/SinaNewsRawData.csv",header=resultList.columns)
# resultList

In [5]:
resultList=pd.read_csv("newsData/SinaNewsRawData.csv")
resultList

,Unnamed: 0,link,title,date,source,article
0,0,https://finance.sina.com.cn/roll/2023-05-12/do...,4月居民存款锐减之谜，1.2万亿去哪了？,2023年05月12日 16:48,第一财经,炒股就看金麒麟分析师研报，权威，专业，及时，全面，助您挖掘潜力主题机会！
1,1,https://finance.sina.com.cn/roll/2023-05-12/do...,4月居民存款锐减之谜，1.2万亿去哪了？,2023年05月12日 16:48,第一财经,但这在数据上并未得到印证。国家统计局公布的数据显示，4月份，全国CPI同比上涨0.1%；...
2,2,https://finance.sina.com.cn/roll/2023-05-12/do...,4月居民存款锐减之谜，1.2万亿去哪了？,2023年05月12日 16:48,第一财经,4月居民存款大幅减少引起市场的广泛热议。
3,3,https://finance.sina.com.cn/roll/2023-05-12/do...,4月居民存款锐减之谜，1.2万亿去哪了？,2023年05月12日 16:48,第一财经,央行最新公布的4月金融数据显示，4月人民币存款减少4609亿元，同比多减5524亿元，其...
4,4,https://finance.sina.com.cn/roll/2023-05-12/do...,4月居民存款锐减之谜，1.2万亿去哪了？,2023年05月12日 16:48,第一财经,不少分析认为，4月居民存款减少或源自“提前还贷”和消费回升所致；另外的观点则认为，4月份...
...,...,...,...,...,...,...
47350,2,https://finance.sina.com.cn/tob/2023-05-10/doc...,蒙牛乳业5月9日斥资约1826.08万港元回购55万股,2023年05月10日 08:59,新浪港股,责任编辑：卢昱君
47351,0,https://finance.sina.com.cn/tob/2023-05-10/doc...,太古股份公司A5月9日斥资272.95万港元回购4.55万股,2023年05月10日 08:58,新浪港股,太古股份公司A（00019）发布公告，于2023年5月9日斥资272.95万港元回购4....
47352,1,https://finance.sina.com.cn/tob/2023-05-10/doc...,太古股份公司A5月9日斥资272.95万港元回购4.55万股,2023年05月10日 08:58,新浪港股,责任编辑：卢昱君
47353,0,https://finance.sina.com.cn/tob/2023-05-10/doc...,太古股份公司B5月9日耗资约550.67万港元回购55.5万股,2023年05月10日 08:58,新浪港股,太古股份公司B（00087）公布，2023年5月9日耗资约550.67万港元回购55.5...


### 1.2 新闻联播文本稿

### 1.3 微博基于关键字内容爬取 (主要内容: 文章文本，观看人数，评论，点赞)

### 1.4 微信公众号基于关键字爬取 (主要内容: 文章文本，评论，点赞，收藏)

### 1.5 卫生办 新冠病毒 人数统计(日)

In [6]:
class newsToolForHealthCenter:
    @staticmethod
    def get_page_news(keyword:str,browser):
        newsList=[]
        news = browser.find_elements("xpath",'//*[@id="0"]/div[1]/a')
        if len(news)==0:
            return np.nan
        for i in news:
            link = i.get_attribute('href') #得到新闻url
            # print(len(urlList),link)
        return newsList

In [7]:
from selenium.common import NoSuchElementException

#打开浏览器
UrlList=[]
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
browser.implicitly_wait(10)
#打开网址
browser.get('http://zs.kaipuyun.cn/s?token=&siteCode=N000001642&searchWord=%E6%96%B0%E5%9E%8B%E5%86%A0%E7%8A%B6%E7%97%85%E6%AF%92&button=')
#获取当前页面新闻的url
UrlList=[]
currentPageList=newsToolForHealthCenter.get_page_news("截至???新型冠状病毒肺炎疫情最新情况",browser)
if currentPageList is not np.nan:
    print(currentPageList)

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 114
Current browser version is 116.0.5845.111 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00E4A813+48355]
	(No symbol) [0x00DDC4B1]
	(No symbol) [0x00CE5358]
	(No symbol) [0x00D061AC]
	(No symbol) [0x00D01EF3]
	(No symbol) [0x00D00579]
	(No symbol) [0x00D30C55]
	(No symbol) [0x00D3093C]
	(No symbol) [0x00D2A536]
	(No symbol) [0x00D082DC]
	(No symbol) [0x00D093DD]
	GetHandleVerifier [0x010AAABD+2539405]
	GetHandleVerifier [0x010EA78F+2800735]
	GetHandleVerifier [0x010E456C+2775612]
	GetHandleVerifier [0x00ED51E0+616112]
	(No symbol) [0x00DE5F8C]
	(No symbol) [0x00DE2328]
	(No symbol) [0x00DE240B]
	(No symbol) [0x00DD4FF7]
	BaseThreadInitThunk [0x771400C9+25]
	RtlGetAppContainerNamedObjectPath [0x77847B1E+286]
	RtlGetAppContainerNamedObjectPath [0x77847AEE+238]
